In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sys
sys.path.insert(1, '../Config')
from lablib import LS_fit

# Prima parte

## Circuito RC

Abbiamo effettuato le misurazioni con il cursore dell'oscilloscopio. \
**Resistenza interna oscilloscopio** = 50 $\Omega$

Intrvallo tensione (0.5,-0.5) V \
Capacità = 10 nF \
resistenza = 67.1 $k\Omega$ \
frequenza = 200 Hz \
incertezza su oscilloscopio (precisione strumento) = 8 mV


In [2]:
# Primo plot dei dati
dati1 = pd.read_csv("Dati1.csv")

tempi1 = dati1["tempo(carica)[ms]"]
tens1 = dati1["tensione(carica1)[mV]"]

# plt.errorbar(tempi1, tens1, [8 for i in tens1], fmt = '.')
# plt.show()

In [3]:
# prova fit dei dati con libreria
def mod_exp(x, A, T, C):
    return A*np.exp(-x/T) + C

tempi1_µs = [i*1e-3 for i in tempi1]
tens1_mv = [i*1e-3 for i in tens1]
tens1_err = [.008 for i in tens1]

par1, val1, ers1, pval1, *other = LS_fit(tempi1_µs, tens1_mv, tens1_err, mod_exp, disp=1, A=1, T=1, C=0)

for p, v, e in zip(par1, val1, ers1):
    print(f"{p} = {v:.3f} ± {e:.3f}")

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 2.716 (χ²/ndof = 0.2)      │              Nfcn = 157              │
│ EDM = 1.58e-06 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ A    │   0.964   │   0.006   │            │            │         │         │       │
│ 1 │ T    │ 0.652e-3  │ 0.011e-3  │            │            │         │         │       │
│ 2 │ C    │  -0.002   │   0.005   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────────────────┐
│   │         A         T         C │
├───┼───────────────────────────────┤
│ A │  3.92e-05   1.36e-9 -0.013e-3 │
│ T │   1.36e-9  1.21e-10 -42.41e-9 │
│ C │ -0.013e-3 -42.41e-9  2.25e-05 │
└───┴───────────────────────────────┘

A = 0.964 ± 0.006
T = 0.001 ± 0.000
C = -0.002 ± 0.005


In [9]:
QC, tau_1, c_0 = val1
R_1 = 67.1e3 # Ohm

C = tau_1/R_1
print(f"Capacità calcolata = {C:.3g} F")

Capacità calcolata = 9.72e-09 F


## Circuito RL

induttanza circa 10 mH \
resistenza = 1 $k\Omega$ \
frequenza = 700 Hz \
range di tensione = -1,1 V \
precisione oscilloscopio = 20 mV

In [5]:
# plot dei dati

dati2 = pd.read_csv("Dati2.csv")
tempi2 = dati2["tempo[ms]"]
tens2 = dati2["tensione[mV](carica)"]

# plt.errorbar(tempi2, tens2, [20 for i in tens2])
# plt.show()

In [6]:
# Fit circuito RL

tempi2_ms = [i*1e-3 for i in tempi2]
tens2_mv = [i*1e-3 for i in tens2]
tens2_err = [.02 for i in tens2]

par2, val2, ers2, pval2, *other = LS_fit(tempi2_ms, tens2_mv, tens2_err, mod_exp, disp=1, A=1, T=1, C=1000)

for p, v, e in zip(par2, val2, ers2):
    print(f"{p} = {v:.3g} ± {e:.3g}")

c:\Users\i\AppData\Local\Programs\Python\Python311\Lib\site-packages\iminuit\cost.py:130: RuntimeWarning: overflow encountered in multiply
  return z * z


┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 17.82 (χ²/ndof = 1.0)      │              Nfcn = 267              │
│ EDM = 2.97e-08 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬──────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼──────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ A    │  -1.873   │   0.016   │            │            │         │         │       │
│ 1 │ T    │  52.2e-6  │  1.1e-6   │            │            │         │         │       │
│ 2 │ C    │   0.967   │   0.010   │            │            │         │         │       │
└───┴──────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌───┬───────────────────────────────┐
│   │         A         T         C │
├───┼───────────────────────────────┤
│ A │  0.000248 1.7845e-9  -0.05e-3 │
│ T │ 1.7845e-9  1.14e-12 8.3430e-9 │
│ C │  -0.05e-3 8.3430e-9  9.92e-05 │
└───┴───────────────────────────────┘

A = -1.87 ± 0.0158
T = 5.22e-05 ± 1.07e-06
C = 0.967 ± 0.00996


In [8]:
Va, tau_2, c_1 = val2
R_2 = 1e3 # ohm

L = R_2 * tau_2
print(f"Induttanza misurata = {L:.3g} H")

Induttanza misurata = 0.0522 H


 # Seconda parte

## Circuito RLC

Resistenza = 300 $\Omega$ \
Induttanza e capacità uguali a prima \
Intervallo tensione (1,0)

Abbiamo stimato le costanti del sistema prima di prendere i dati:
* $\omega_0$ = 100 kHz
* $f_0 = \frac{\omega_0}{2\pi}$ = 16kHz 

Ci siamo resi presto conto che questi risultati sono insensati \
Frequenza effettivamente usata = 300 Hz


Per smorzamento critico abbiamo usato Resistenza = 3900 $\Omega$ \
Per sovrasmorzamento abbiamo usato Resistenza = 10.000 $\Omega$

### Caso sottosmorzato

In [ ]:
# fit dei dati nel caso sottosmorzato



### Caso criticamente smorzato

### Caso sovrasmorzato